In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
# Collect 1% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_1, _ , y_1 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.01, stratify=y_label_0_1, random_state=42)

In [6]:
X_1.shape

(9118, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt1 = DecisionTreeClassifier()
dt1.fit(X_1, y_1)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [  2400      9   1421    364      2    949     81 108734      8      3]
(2400, 56) (2400,)
Save 0 to baseline_Def1.npy
(9, 56) (9,)
Save 1 to baseline_Def2.npy
(1421, 56) (1421,)
Save 2 to baseline_Def3.npy
(364, 56) (364,)
Save 3 to baseline_Def4.npy
(2, 56) (2,)
Save 4 to baseline_Def5.npy
(949, 56) (949,)
Save 5 to baseline_Def6.npy
(81, 56) (81,)
Save 7 to baseline_Def8.npy
(108734, 56) (108734,)
Save 8 to baseline_Def9.npy
(8, 56) (8,)
Save 9 to baseline_Def10.npy
(3, 56) (3,)
Save 10 to baseline_Def11.npy
Execution Time: 0.392374 seconds
(341913,)
[0 1 2 3 4 5 7 8 9] [162651     47    446   1394    396 137087     50  39839      3]
(162651, 56) (162651,)
Save 0 to BIM_Def1.npy
(47, 56) (47,)
Save 1 to BIM_Def2.npy
(446, 56) (446,)
Save 2 to BIM_Def3.npy
(1394, 56) (1394,)
Save 3 to BIM_Def4.npy
(396, 56) (396,)
Save 4 to BIM_Def5.npy
(137087, 56) (137087,)
Save 5 to BIM_Def6.npy
(50, 56) (50,)
Save 7 to BIM_Def8.npy
(39839, 56) (39839,)
Sa

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [  1912     16   1417    142      1    366     23 110086      7      1]
(1912, 56) (1912,)
Save 0 to baseline_Def1.npy
(16, 56) (16,)
Save 1 to baseline_Def2.npy
(1417, 56) (1417,)
Save 2 to baseline_Def3.npy
(142, 56) (142,)
Save 3 to baseline_Def4.npy
(1, 56) (1,)
Save 4 to baseline_Def5.npy
(366, 56) (366,)
Save 5 to baseline_Def6.npy
(23, 56) (23,)
Save 7 to baseline_Def8.npy
(110086, 56) (110086,)
Save 8 to baseline_Def9.npy
(7, 56) (7,)
Save 9 to baseline_Def10.npy
(1, 56) (1,)
Save 10 to baseline_Def11.npy
Execution Time: 0.846087 seconds
(341913,)
[0 2 3 5 8] [146438      1     66 184156  11252]
(146438, 56) (146438,)
Save 0 to BIM_Def1.npy
(1, 56) (1,)
Save 2 to BIM_Def3.npy
(66, 56) (66,)
Save 3 to BIM_Def4.npy
(184156, 56) (184156,)
Save 5 to BIM_Def6.npy
(11252, 56) (11252,)
Save 8 to BIM_Def9.npy
Execution Time: 2.109127 seconds
(341913,)
[0 3 5 8] [131753     33 199653  10474]
(131753, 56) (131753,)
Save 0 to FGSM_Def1.npy
(33, 56

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ExcludeCaFA/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ExcludeCaFA/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [ 7062   131  2921  1612    22  2739   471 98802   123    88]
(7062, 56) (7062,)
Save 0 to baseline_Def1.npy
(131, 56) (131,)
Save 1 to baseline_Def2.npy
(2921, 56) (2921,)
Save 2 to baseline_Def3.npy
(1612, 56) (1612,)
Save 3 to baseline_Def4.npy
(22, 56) (22,)
Save 4 to baseline_Def5.npy
(2739, 56) (2739,)
Save 5 to baseline_Def6.npy
(471, 56) (471,)
Save 7 to baseline_Def8.npy
(98802, 56) (98802,)
Save 8 to baseline_Def9.npy
(123, 56) (123,)
Save 9 to baseline_Def10.npy
(88, 56) (88,)
Save 10 to baseline_Def11.npy
Execution Time: 0.191111 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [ 96548   2280   2122   9054   2175 148569   9934  70963    107    161]
(96548, 56) (96548,)
Save 0 to BIM_Def1.npy
(2280, 56) (2280,)
Save 1 to BIM_Def2.npy
(2122, 56) (2122,)
Save 2 to BIM_Def3.npy
(9054, 56) (9054,)
Save 3 to BIM_Def4.npy
(2175, 56) (2175,)
Save 4 to BIM_Def5.npy
(148569, 56) (148569,)
Save 5 to BIM_Def6.npy
(9934, 56) (9934,)
Save 7 to B

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ExcludeCaFA/UNSW_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_ExcludeCaFA.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB1,2400,9,1421,364,2,949,0,81,108734,8,3,113971
1,XGB1,162651,47,446,1394,396,137087,0,50,39839,3,0,341913
2,XGB1,134425,22,550,470,259,166821,0,119,39244,3,0,341913
3,XGB1,162651,47,446,1394,396,137087,0,50,39839,3,0,341913
4,XGB1,78278,1009,2076,234,13,240312,436,15783,3766,0,6,341913
5,XGB1,133204,36,2514,1502,370,132688,0,141,71440,12,6,341913
6,XGB1,10084,15,3732,1074,6,3048,0,246,323678,21,9,341913
7,XGB1,40515,0,321,529,2,1277,0,83,299182,2,2,341913
8,XGB1,161582,113,110,1071,210,148129,0,15,30683,0,0,341913
9,XGB1,159559,31,1000,630,634,145154,0,175,34728,2,0,341913
